In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load 4 plasmid titration experiments, 2 each that do/don't change marker levels

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
exp100_path = base_path/'2024.07.18_exp100'/'export'
exp100_2_path = base_path/'2024.08.16_exp100.2'/'export'

plates = pd.DataFrame({
    'data_path': [exp100_path, exp100_2_path, base_path/'2024.09.11_exp100.3'/'export', base_path/'2024.09.14_exp100.4'/'export', base_path/'2024.09.28_exp100.5'/'export'],
    'yaml_path': [exp100_path/'wells.yaml', exp100_2_path/'wells.yaml']*2 + [exp100_path/'wells.yaml'],
    'biorep': [1, 2]*2 + [3],
    'exp': ['exp100.' + str(i) for i in range(1,6)],
})

output_path = rd.rootdir/'output'/'plasmid-titration'
cache_path = output_path/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mRuby2-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]

    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

In [ ]:
data['marker'] = data['mGL-A']
data['output'] = data['mRuby2-A']
data['filler'] = data['tagBFP-A']

In [ ]:
no_yellow_viridis = matplotlib.colors.ListedColormap(matplotlib.colormaps['viridis'](np.linspace(0,0.82,256)))

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A', 'tagBFP-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gate for exp100
gates.loc[len(gates.index)] = ['exp100.1', gates['mGL-A'].mean(), gates['mRuby2-A'].mean(), gates['tagBFP-A'].mean()] 

# Indicate which channels are relevant for each experiment
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'
gates['filler'] = 'tagBFP-A'
gates['marker'] = gates['mGL-A']
gates['output'] = gates['mRuby2-A']
gates['filler'] = gates['tagBFP-A']

display(gates)

In [ ]:
plot_df = data.groupby(['exp','construct','amount']).sample(1000)
g = sns.displot(data=plot_df, x='marker', y='output', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=5)


In [ ]:
plot_df = data.groupby(['exp','construct','amount']).sample(1000)
g = sns.displot(data=plot_df, x='marker', y='filler', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=5)


In [ ]:
plot_df = data
g = sns.displot(data=plot_df, x='marker', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False)
g.set(xlim=(2e0,1e5))

In [ ]:
plot_df = data
g = sns.displot(data=plot_df, x='filler', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False)
g.set(xlim=(2e0,1e6))

In [ ]:
plot_df = data
g = sns.displot(data=plot_df, x='output', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False)
g.set(xlim=(2e0,1e6))

In [ ]:
# Categorize cells into quadrants based on two gates
# Possible values:
#   0 = double negative
#   1 = x-positive
#   2 = y-positive
#   3 = double positive
def get_quadrant(x,y,gate_x,gate_y):
    df_quad = pd.DataFrame()
    df_quad['x'] = x > gate_x
    df_quad['y'] = y > gate_y
    df_quad['quadrant'] = df_quad['x'].astype(int) + df_quad['y'].astype(int)*2
    return df_quad['quadrant']

df_list = []
for exp, group in data.groupby('exp'): 
    df = group.copy()
    df['quadrant'] = get_quadrant(group['marker'], group['filler'], 
                                  gates.loc[gates['exp']==exp, 'marker'].values[0],
                                  gates.loc[gates['exp']==exp, 'filler'].values[0],)
    df_list.append(df)
df = pd.concat(df_list)
display(df)

In [ ]:
df_transfected = df[(df['quadrant']>0) & (df['construct']!='UT')]
df_gated = df[(df['quadrant'].isin([1,3])) & (df['construct']!='UT')]

In [ ]:
plot_df = df_transfected
g = sns.displot(data=plot_df, x='output', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False)
g.set(xlim=(2e0,1e6))

In [ ]:
plot_df = df_transfected.groupby(['exp','construct','amount']).sample(1000)
g = sns.displot(data=plot_df, x='marker', y='output', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=5)

In [ ]:
plot_df = df_gated
g = sns.displot(data=plot_df, x='marker', y='output', hue='amount', palette=no_yellow_viridis, kind='kde',
                hue_norm=matplotlib.colors.LogNorm(), row='exp', col='construct', facet_kws=dict(margin_titles=True),
                log_scale=True, common_norm=False, levels=5)

### Plot data for experiments with constant marker
(exp100.1, 100.3, 100.5)

In [ ]:
# Bin data and calculate statistics
df_quantiles, stats, _, fits = base.calculate_bins_stats(df_gated, by=['construct','amount','exp','biorep'])
stats = stats.merge(metadata, how='left', on='construct')
fits = fits.merge(metadata, how='left', on='construct')

In [ ]:
plot_df = df_quantiles[(df_quantiles['construct']!='AIO.039') & (df_quantiles['exp']=='exp100.1')]
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             col='amount')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
marker_baseline = stats.loc[(stats['exp']=='exp100.1') & (stats['construct']=='AIO.039'), 'output_gmean'].mean()
for _, ax in g.axes_dict.items(): ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
g.figure.savefig(rd.outfile(output_path/'line_exp100_by-amount.svg'))

In [ ]:
plot_df = df_quantiles[(df_quantiles['construct']!='AIO.039') & (df_quantiles['exp']=='exp100.3')]
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             col='amount')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
marker_baseline = stats.loc[(stats['exp']=='exp100.3') & (stats['construct']=='AIO.039'), 'output_gmean'].mean()
for _, ax in g.axes_dict.items(): ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
g.figure.savefig(rd.outfile(output_path/'line_exp100.3_by-amount.svg'))

In [ ]:
plot_df = df_quantiles[(df_quantiles['construct']!='AIO.039') & (df_quantiles['exp']=='exp100.5')]
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             col='amount')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
marker_baseline = stats.loc[(stats['exp']=='exp100.5') & (stats['construct']=='AIO.039'), 'output_gmean'].mean()
for _, ax in g.axes_dict.items(): ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
g.figure.savefig(rd.outfile(output_path/'line_exp100.5_by-amount.svg'))

In [ ]:
label = {
    'RC140': 'base',
    'RC142': 'OL circuit',
    'RC143': 'CL circuit',
}
df_quantiles['label'] = df_quantiles['construct'].map(label)
stats['label'] = stats['construct'].map(label)
fits['label'] = fits['construct'].map(label)

In [ ]:
plot_df = df_quantiles[(df_quantiles['construct']!='AIO.039') & (df_quantiles['exp']=='exp100.1')].sort_values('construct')
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='amount', palette=no_yellow_viridis, 
                hue_norm=matplotlib.colors.LogNorm(),
                legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
                estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
                col='label')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
g.set_titles('{col_name}')
marker_baseline = stats.loc[(stats['exp']=='exp100.1') & (stats['construct']=='AIO.039'), 'output_gmean'].mean()
for _, ax in g.axes_dict.items(): ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
g.figure.savefig(rd.outfile(output_path/'line_exp100_by-construct.svg'))

In [ ]:
plot_df = df_quantiles[(df_quantiles['construct']!='AIO.039') & (df_quantiles['exp']=='exp100.3')].sort_values('construct')
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='amount', palette=no_yellow_viridis, 
                hue_norm=matplotlib.colors.LogNorm(),
                legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
                estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
                col='label')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
g.set_titles('{col_name}')
marker_baseline = stats.loc[(stats['exp']=='exp100.3') & (stats['construct']=='AIO.039'), 'output_gmean'].mean()
for _, ax in g.axes_dict.items(): ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
g.figure.savefig(rd.outfile(output_path/'line_exp100.3_by-construct.svg'))

In [ ]:
plot_df = df_quantiles[(df_quantiles['construct']!='AIO.039') & (df_quantiles['exp']=='exp100.5')].sort_values('construct')
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='amount', palette=no_yellow_viridis, 
                hue_norm=matplotlib.colors.LogNorm(),
                legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
                estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
                col='label')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
g.set_titles('{col_name}')
marker_baseline = stats.loc[(stats['exp']=='exp100.5') & (stats['construct']=='AIO.039'), 'output_gmean'].mean()
for _, ax in g.axes_dict.items(): ax.axhline(marker_baseline, color='black', ls=':', zorder=0)
g.figure.savefig(rd.outfile(output_path/'line_exp100.5_by-construct.svg'))

In [ ]:
plot_df = fits[(fits['construct']!='AIO.039') & (fits['exp']=='exp100.1')].sort_values('construct')

fig, axes = plt.subplots(1,2, figsize=(10,5), sharey=True,)
sns.scatterplot(data=plot_df, x='label', y='slope', hue='amount', palette=no_yellow_viridis, 
                hue_norm=matplotlib.colors.LogNorm(), legend=False, ax=axes[0])
axes[0].set(xlabel='', ylim=(0,1.2), xlim=(-0.5,2.5))

sns.scatterplot(data=plot_df, x='amount', y='slope', hue='construct', palette=main_palette,
                style='construct', markers=main_markers, legend=False, ax=axes[1])
axes[1].set(ylim=(0,1.2))
sns.despine()

fig.savefig(rd.outfile(output_path/'scatter_exp100_slopes.svg'), bbox_inches='tight')

In [ ]:
plot_df = fits[(fits['construct']!='AIO.039') & (fits['exp'].isin(['exp100.1', 'exp100.3', 'exp100.5']))].sort_values('construct')

fig, axes = plt.subplots(1,2, figsize=(10,5), sharey=True,)
sns.stripplot(data=plot_df, x='label', y='slope', hue='amount', palette=no_yellow_viridis, 
                hue_norm=matplotlib.colors.LogNorm(), legend=False, ax=axes[0], dodge=True, jitter=False,
                size=9, linewidth=1, edgecolor='white')
axes[0].set(xlabel='', ylim=(0,1.2), xlim=(-0.5,2.5))

sns.scatterplot(data=plot_df, x='amount', y='slope', hue='construct', palette=main_palette,
                style='construct', markers=main_markers, legend=False, ax=axes[1])
axes[1].set(ylim=(0,1.2))
sns.despine()

fig.savefig(rd.outfile(output_path/'scatter_slopes.svg'), bbox_inches='tight')

### Explore experiments where marker was also titrated 
(exp100.2, 100.4)

In [ ]:
# Bin data and calculate statistics
df_quantiles2, stats2, _, fits2 = base.calculate_bins_stats(df_transfected)
stats2 = stats2.merge(metadata, how='left', on='construct')
fits2 = fits2.merge(metadata, how='left', on='construct')

df_quantiles2['label'] = df_quantiles2['construct'].map(label)

In [ ]:
plot_df = df_quantiles2[(df_quantiles2['construct']!='AIO.039') & (df_quantiles2['exp']=='exp100.2')]
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             col='amount')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
g.figure.savefig(rd.outfile(output_path/'line_exp100.2_by-amount.svg'))

In [ ]:
plot_df = df_quantiles2[(df_quantiles2['construct']!='AIO.039') & (df_quantiles2['exp']=='exp100.2')].sort_values('construct')
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='amount', palette=no_yellow_viridis, 
                hue_norm=matplotlib.colors.LogNorm(),
                legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
                estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
                col='label')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
g.set_titles('{col_name}')
g.figure.savefig(rd.outfile(output_path/'line_exp100.2_by-construct.svg'))

In [ ]:
# Bin data and calculate statistics
df_quantiles_ungated, stats_ungated, _, fits_ungated = base.calculate_bins_stats(df)
stats_ungated = stats_ungated.merge(metadata, how='left', on='construct')
fits_ungated = fits_ungated.merge(metadata, how='left', on='construct')

df_quantiles_ungated['label'] = df_quantiles_ungated['construct'].map(label)

In [ ]:
plot_df = df_quantiles_ungated[(df_quantiles_ungated['construct']!='AIO.039') & (df_quantiles_ungated['exp']=='exp100.2')]
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
             col='amount')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
g.figure.savefig(rd.outfile(output_path/'line_exp100.2_by-amount.svg'))

In [ ]:
plot_df = df_quantiles_ungated[(df_quantiles_ungated['construct']!='AIO.039') & (df_quantiles_ungated['exp']=='exp100.2')].sort_values('construct')
g = sns.relplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='amount', palette=no_yellow_viridis, 
                hue_norm=matplotlib.colors.LogNorm(),
                legend=False, dashes=False, style='construct', markers=main_markers, kind='line', markersize=10, markeredgewidth=1,
                estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)),
                col='label')
g.set(xscale='log', yscale='log', xlabel='marker', ylim=(2e1,1e5))
g.set_titles('{col_name}')
g.figure.savefig(rd.outfile(output_path/'line_exp100.2_by-construct.svg'))